# Подготовка окружения

## Установка пакетов и импорт зависимостей

In [1]:
!pip install datasets
!pip install pandas
!pip install plotly
!pip install accelerate -U
!pip install transformers[torch] -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.1 MB/s eta 0:00:00


In [2]:
import sys
import importlib
import os
import random
from multiprocessing import cpu_count
from getpass import getpass

import logging

from typing import Tuple
from google.colab import drive
from joblib import dump, load

# import faiss
import torch
from transformers import AutoTokenizer, GPT2Tokenizer

## Настройка окружения

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
dump(tokenizer, '/content/drive/MyDrive/docs/keepForever/mipt/nlp/hw_4sem/code/ml/asset/tokenizer/gpt2_tokenizer.joblib')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

['/content/drive/MyDrive/docs/keepForever/mipt/nlp/hw_4sem/code/ml/asset/tokenizer/gpt2_tokenizer.joblib']

## Клонирование исходных кодов проекта

In [30]:
def clone_pull_github_src(pull: bool = True):
    """
    Клонирует или обновляет репозиторий GitHub в локальный каталог для последующей работы.

    Parameters:
    pull (bool): Указывает, следует ли выполнять pull для существующего репозитория. Если True, выполняется git pull.
                 Если False, репозиторий клонируется в указанный каталог.

    Returns:
    Constants: Экземпляр класса Constants, содержащий константы проекта.
    """
    WORKSPACE_PATH = '/content/drive/MyDrive/docs/keepForever/mipt/nlp/hw_4sem/'
    WORKSPACE_TMP = WORKSPACE_PATH + '/tmp/'
    GIT_HUB_PROJECT_PATH = WORKSPACE_PATH + 'code/'

    token = getpass('Введите GitHub token: ')
    repo_url = 'https://github.com/km-mipt-nlp-gen/hw2.git'
    repo_url_with_token = repo_url.replace('https://', f'https://{token}@')

    os.chdir(GIT_HUB_PROJECT_PATH)

    if pull:
        !git pull origin main
    else:
        !git clone {repo_url_with_token} "$GIT_HUB_PROJECT_PATH"

    del token

    sys.path.append(f"{GIT_HUB_PROJECT_PATH}/web_app/src/")
    import constants_module
    importlib.reload(constants_module)
    from constants_module import Constants

    return Constants()

constants = clone_pull_github_src()
from constants_module import Constants
from chat_util_module import ChatUtil

Введите GitHub token: ··········
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 6 (delta 4), reused 6 (delta 4), pack-reused 0
Unpacking objects: 100% (6/6), 476 bytes | 6.00 KiB/s, done.
From https://github.com/km-mipt-nlp-gen/hw2
 * branch            main       -> FETCH_HEAD
   439ef59..584484d  main       -> origin/main
Updating 439ef59..584484d
Fast-forward
 ml/src/train/gpt2_fine_tuning_pipeline_module.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)
DEVICE: cuda:0
Число процессов для использования: 12


# Тренировка моделей и создание ембеддингов

## Обновление модулей

In [31]:
def reload_modules(constants: Constants) -> Tuple[Constants, ChatUtil]:
    """
    Перезагружает модули проекта для обновления изменений в коде без перезапуска среды выполнения.

    Parameters:
    constants (Constants): Экземпляр класса Constants, содержащий постоянные поля.

    Returns:
    Tuple[Constants, ChatUtil]: Кортеж, содержащий обновленный экземпляр класса Constants и экземпляр ChatUtil.
    """
    import sys
    sys.path.append(f"{constants.GIT_HUB_PROJECT_PATH}/web_app/src/")
    sys.path.append(f"{constants.GIT_HUB_PROJECT_PATH}/ml/src/train/")

    import importlib
    import chat_util_module
    import constants_module
    import models_zoo_module
    import gpt2_fine_tuning_pipeline_module

    importlib.reload(chat_util_module)
    importlib.reload(constants_module)
    importlib.reload(models_zoo_module)
    importlib.reload(gpt2_fine_tuning_pipeline_module)

    from constants_module import Constants
    from chat_util_module import ChatUtil
    from gpt2_fine_tuning_pipeline_module import GPT2TrainingPipeline
    from models_zoo_module import GPT2

    constants = Constants()

    return constants, ChatUtil(logging.DEBUG, constants)

constants, chat_util = reload_modules(constants)
preprocessed_data = load(constants.PROCESSED_QA_PATH)

DEVICE: cuda:0
Число процессов для использования: 12


## Тренировка моделей

### Тренировка GPT2 (fine-tuning)

In [32]:
from gpt2_fine_tuning_pipeline_module import GPT2TrainingPipeline

In [33]:
gpt2_training_pipeline = GPT2TrainingPipeline(preprocessed_data[:100], constants, chat_util)

In [34]:
gpt2_model, losses = gpt2_training_pipeline.train()

Step,Training Loss
1,9.490500
2,9.219400
3,9.899000
4,9.264600
5,9.256400
6,9.396500
7,9.093700
8,9.732600
9,9.097100
10,9.282900


DEBUG:chat_util_module:Запись потерь {'loss': 9.4905, 'grad_norm': nan, 'learning_rate': 0.0, 'epoch': 0.02} ..
DEBUG:chat_util_module:Потери: 9.4905
DEBUG:chat_util_module:Запись потерь {'loss': 9.2194, 'grad_norm': nan, 'learning_rate': 0.0, 'epoch': 0.04} ..
DEBUG:chat_util_module:Потери: 9.2194
DEBUG:chat_util_module:Запись потерь {'loss': 9.899, 'grad_norm': inf, 'learning_rate': 0.0, 'epoch': 0.06} ..
DEBUG:chat_util_module:Потери: 9.899
DEBUG:chat_util_module:Запись потерь {'loss': 9.2646, 'grad_norm': inf, 'learning_rate': 0.0, 'epoch': 0.08} ..
DEBUG:chat_util_module:Потери: 9.2646
DEBUG:chat_util_module:Запись потерь {'loss': 9.2564, 'grad_norm': inf, 'learning_rate': 0.0, 'epoch': 0.1} ..
DEBUG:chat_util_module:Потери: 9.2564
DEBUG:chat_util_module:Запись потерь {'loss': 9.3965, 'grad_norm': inf, 'learning_rate': 0.0, 'epoch': 0.12} ..
DEBUG:chat_util_module:Потери: 9.3965
DEBUG:chat_util_module:Запись потерь {'loss': 9.0937, 'grad_norm': inf, 'learning_rate': 0.0, 'epoch': 

## Проверка тренировки

In [35]:
gpt2_model

GPT2(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [36]:
losses

[9.4905,
 9.2194,
 9.899,
 9.2646,
 9.2564,
 9.3965,
 9.0937,
 9.7326,
 9.0971,
 9.2829,
 8.6682,
 7.518,
 6.8565,
 5.5122,
 3.5662,
 1.9686,
 0.9336,
 0.9576,
 0.598,
 0.4294,
 0.3773,
 0.5011,
 0.4149,
 0.5405,
 0.6337,
 0.5371,
 0.5319,
 0.4676,
 0.3575,
 0.2251,
 0.6194,
 0.4732,
 0.674,
 0.2702,
 0.4101,
 0.2231,
 0.3577,
 0.2133,
 0.6482,
 0.2171,
 0.7546,
 0.3529,
 0.4231,
 0.5634,
 0.9731,
 0.3194,
 0.6272,
 0.3326,
 0.383,
 0.1829]

In [59]:
from transformers import AutoModel, GPT2LMHeadModel

gpt_model = GPT2LMHeadModel.from_pretrained(constants.GPT_MODEL_OUTPUT_DIR).to(constants.DEVICE)
gpt_tokenizer = constants.GPT_TOKENIZER

def generate_text(model, tokenizer, constants, sequence, max_length, num_return_sequences=1, top_k=50, top_p=0.95):
    input_ids = tokenizer.encode(sequence, return_tensors='pt').to(constants.DEVICE)

    generated_text_ids = model.generate(
        input_ids,
        do_sample=True,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        pad_token_id=model.config.eos_token_id,
        top_k=top_k,
        top_p=top_p,
    )

    generated_text = tokenizer.decode(generated_text_ids[0], skip_special_tokens=True)
    return generated_text

quote = 'R_3: "Bart Simpson: Good one, Dad."; R_2: "Homer Simpson: Okay, kids. Prepare to be dazzled. (CALLING) Marge! Turn on the juice!"; R_1: "Homer Simpson: What do you think, kids?"'
generate_text(gpt_model, gpt_tokenizer, constants, quote, max_length=1000, top_k=50, top_p=0.95)

'R_3: "Bart Simpson: Good one, Dad."; R_2: "Homer Simpson: Okay, kids. Prepare to be dazzled. (CALLING) Marge! Turn on the juice!"; R_1: "Homer Simpson: What do you think, kids?"; R_0: "Bart Simpson: I can\'t think of a better one for my family than this!"'